#### Imports

In [ ]:
import pandas as pd
import boto3
import requests
import geopandas as gpd
import gzip
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from IPython.display import display
from shapely.geometry import Point
from scipy.spatial.distance import cdist
from geopy.distance import geodesic


#### Data 

In [19]:
bike_data = pd.read_csv("202409-citibike-tripdata/202409-citibike-tripdata/202409-citibike-tripdata_1.csv.zip", compression="zip")
bike_data.info()

/tmp/ipykernel_3051480/3236835982.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  bike_data = pd.read_csv("202409-citibike-tripdata/202409-citibike-tripdata/202409-citibike-tripdata_1.csv.zip", compression="zip")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ride_id             1000000 non-null  object 
 1   rideable_type       1000000 non-null  object 
 2   started_at          1000000 non-null  object 
 3   ended_at            1000000 non-null  object 
 4   start_station_name  999566 non-null   object 
 5   start_station_id    999566 non-null   object 
 6   end_station_name    999885 non-null   object 
 7   end_station_id      999768 non-null   object 
 8   start_lat           1000000 non-null  float64
 9   start_lng           1000000 non-null  float64
 10  end_lat             999991 non-null   float64
 11  end_lng             999991 non-null   float64
 12  member_casual       1000000 non-null  object 
dtypes: float64(4), object(9)
memory usage: 99.2+ MB


In [49]:
stations = bike_data[["start_station_name", "start_station_id", "start_lat", "start_lng"]]
stations = stations.rename(
    columns={
        "start_station_name": "station_name", 
        "start_station_id": "station_id", 
        "start_lat": "station_lat", 
        "start_lng": "station_lng"
    }
)

endstations = bike_data[["end_station_name", "end_station_id", "end_lat", "end_lng"]]
endstations = stations.rename(
    columns={
        "end_station_name": "station_name", 
        "end_station_id": "station_id", 
        "end_lat": "station_lat", 
        "end_lng": "station_lng"
    }
)
stations = pd.concat([stations, endstations])

# Calculate station usage as a feature
station_counts = stations['station_id'].value_counts().to_dict()
stations['station_usage'] = stations['station_id'].map(station_counts).fillna(0).astype(int)

stations = stations.drop_duplicates("station_id")
print(stations.shape)
print(stations.head())

(4245, 5)
               station_name station_id  station_lat  station_lng  \
0       Hudson St & W 13 St    6115.06    40.740057   -74.005274   
1           W 37 St & 5 Ave    6398.06    40.750380   -73.983390   
2  Greenpoint Ave & West St    5752.09    40.729803   -73.959099   
3           E 85 St & 3 Ave    7212.05    40.778012   -73.954071   
4           7 Ave & Park Pl    4125.07    40.677615   -73.973243   

   station_usage  
0           3588  
1           1860  
2           2874  
3           4574  
4           2572  


In [20]:
display(bike_data.head())

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,D86F678648E7A867,electric_bike,2024-09-10 22:50:16.212,2024-09-10 23:30:44.697,Hudson St & W 13 St,6115.06,Broadway & W 58 St,6948.10,40.740057,-74.005274,40.766953,-73.981693,casual
1,032D1788CD512084,electric_bike,2024-09-22 05:51:00.609,2024-09-22 05:56:50.446,W 37 St & 5 Ave,6398.06,9 Ave & W 45 St,6717.06,40.750380,-73.983390,40.760193,-73.991255,member
2,DA55381E5121F0F9,electric_bike,2024-09-24 11:07:40.618,2024-09-24 11:29:23.460,Greenpoint Ave & West St,5752.09,2 Ave & E 72 St,6925.09,40.729803,-73.959099,40.768762,-73.958408,member
3,F67A042C028C6367,classic_bike,2024-09-03 14:25:28.732,2024-09-03 14:33:51.075,E 85 St & 3 Ave,7212.05,2 Ave & E 72 St,6925.09,40.778012,-73.954071,40.768762,-73.958408,member
4,31F722D5EAB9C780,electric_bike,2024-09-09 15:46:50.376,2024-09-09 15:50:16.411,7 Ave & Park Pl,4125.07,Carroll St & 6 Ave,4019.06,40.677615,-73.973243,40.674089,-73.978728,member


In [21]:
stations = bike_data.drop_duplicates(subset="start_station_name", keep="first")
# Prepare coordinates as tuples
coords = stations[['start_lat', 'start_lng']].apply(tuple, axis=1).tolist()

# Calculate pairwise distances in kilometers
distance_matrix = cdist(coords, coords, lambda u, v: geodesic(u, v).km)

print(distance_matrix[distance_matrix > 0].min())

KeyboardInterrupt: 

In [13]:
sorted_distances = np.sort(distance_matrix[distance_matrix > 0])
lowest_20_distances = sorted_distances[:100]

print("Lowest 20 distances:", lowest_20_distances)



Lowest 20 distances: [0.01752215 0.01752215 0.02960589 0.02960589 0.03990461 0.03990461
 0.04019324 0.04019324 0.04085462 0.04085462 0.04291277 0.04291277
 0.05344148 0.05344148 0.05846353 0.05846353 0.06216932 0.06216932
 0.06853825 0.06853825 0.07221339 0.07221339 0.0735768  0.0735768
 0.07584386 0.07584386 0.07667698 0.07667698 0.07716498 0.07716498
 0.07871226 0.07871226 0.07897644 0.07897644 0.07982762 0.07982762
 0.08020803 0.08020803 0.08022998 0.08022998 0.08058615 0.08058615
 0.08356289 0.08356289 0.08380436 0.08380436 0.0849726  0.0849726
 0.08602605 0.08602605 0.08627782 0.08627782 0.08680679 0.08680679
 0.08746055 0.08746055 0.08812281 0.08812281 0.08872965 0.08872965
 0.08974368 0.08974368 0.09004794 0.09004794 0.09056651 0.09056651
 0.09072302 0.09072302 0.09290446 0.09290446 0.09418959 0.09418959
 0.09426648 0.09426648 0.0951635  0.0951635  0.09553263 0.09553263
 0.09905528 0.09905528 0.10066922 0.10066922 0.10147503 0.10147503
 0.10153171 0.10153171 0.10176002 0.1017600

##### Get the NYPD data

In [26]:
nypd_data = pd.read_csv("Motor_Vehicle_Collisions_-_Crashes_20241028.csv")
print(nypd_data.shape)
display(nypd_data.head())
nypd_data.info()

/tmp/ipykernel_3051480/3886911840.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  nypd_data = pd.read_csv("Motor_Vehicle_Collisions_-_Crashes_20241028.csv")


(2129381, 29)


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129381 entries, 0 to 2129380
Data columns (total 29 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   CRASH DATE                     object 
 1   CRASH TIME                     object 
 2   BOROUGH                        object 
 3   ZIP CODE                       object 
 4   LATITUDE                       float64
 5   LONGITUDE                      float64
 6   LOCATION                       object 
 7   ON STREET NAME                 object 
 8   CROSS STREET NAME              object 
 9   OFF STREET NAME                object 
 10  NUMBER OF PERSONS INJURED      float64
 11  NUMBER OF PERSONS KILLED       float64
 12  NUMBER OF PEDESTRIANS INJURED  int64  
 13  NUMBER OF PEDESTRIANS KILLED   int64  
 14  NUMBER OF CYCLIST INJURED      int64  
 15  NUMBER OF CYCLIST KILLED       int64  
 16  NUMBER OF MOTORIST INJURED     int64  
 17  NUMBER OF MOTORIST KILLED      int64  
 18  CO

#### Data Cleaning and Transformation

In [16]:
# drop NaNs and assure that times are datef
bike_data = bike_data.dropna()
# ensure there are no duplicates
bike_data = bike_data.drop_duplicates(subset="ride_id")
# ensure datetime format for time points
bike_data["started_at"] = pd.to_datetime(bike_data['started_at'])
bike_data["ended_at"] = pd.to_datetime(bike_data['ended_at'])

bike_data['start_year'] = bike_data['started_at'].dt.year
bike_data['start_month'] = bike_data['started_at'].dt.month
bike_data['start_day'] = bike_data['started_at'].dt.day
bike_data['start_hour'] = bike_data['started_at'].dt.hour

print(bike_data.shape)

(999338, 17)


In [27]:
# filter NYPD data for bicycle-related accidents
print(nypd_data.shape)
nypd_data = nypd_data[(nypd_data["NUMBER OF CYCLIST INJURED"] > 0) | (nypd_data["NUMBER OF CYCLIST KILLED"] > 0)] 
print(nypd_data.shape)
# ensure crash date is in datetime
nypd_data['CRASH DATE'] = pd.to_datetime(nypd_data['CRASH DATE'])
# get the crash time separated into hour, day, month, year
nypd_data['year'] = nypd_data['CRASH DATE'].dt.year
nypd_data['month'] = nypd_data['CRASH DATE'].dt.month
nypd_data['day'] = nypd_data['CRASH DATE'].dt.day
nypd_data['hour'] = pd.to_datetime(nypd_data['CRASH TIME'], format='%H:%M').dt.hour


(2129381, 29)
(58626, 29)


In [ ]:
stations = bike_data

In [50]:
# Convert CitiBike start and end locations to GeoDataFrames
bike_gdf = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.station_lng, stations.station_lat), crs="EPSG:2263")

# Convert NYPD crash data to GeoDataFrame
nypd_data_spatial = nypd_data.dropna(subset=['LATITUDE', 'LONGITUDE'])
nypd_gdf = gpd.GeoDataFrame(
    nypd_data_spatial,
    geometry=gpd.points_from_xy(nypd_data_spatial['LONGITUDE'], nypd_data_spatial['LATITUDE']),
    crs="EPSG:2263"
)


In [53]:
# Join NYPD data with CitiBike data based on proximity and time
#bike_gdf = bike_gdf.to_crs("EPSG:2263")
#nypd_gdf = nypd_gdf.to_crs("EPSG:2263")
nypd_near_citibike = gpd.sjoin_nearest(nypd_gdf, bike_gdf, max_distance=100)
nypd_near_citibike['accident'] = 1
print(nypd_near_citibike.shape)
display(nypd_near_citibike.head(20))

(108528, 41)


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,day,hour,geometry,index_right,station_name,station_id,station_lat,station_lng,station_usage,accident
27,2021-12-14,12:54,BROOKLYN,11217.0,40.687534,-73.977500,"(40.687534, -73.9775)",FULTON STREET,SAINT FELIX STREET,NaN,...,14,12,POINT (-73.978 40.688),265552,Lafayette Ave & Ft Greene Pl,4470.09,40.687002,-73.976650,2072,1
27,2021-12-14,12:54,BROOKLYN,11217.0,40.687534,-73.977500,"(40.687534, -73.9775)",FULTON STREET,SAINT FELIX STREET,NaN,...,14,12,POINT (-73.978 40.688),594067,Lafayette Ave & Ft Greene Pl,4470.09,40.687002,-73.976650,274,1
31,2021-12-14,16:25,NaN,NaN,40.784615,-73.953964,"(40.784615, -73.953964)",EAST 93 STREET,NaN,NaN,...,14,16,POINT (-73.954 40.785),293115,E 94 St & Madison Ave,7344.01,40.785851,-73.954824,406,1
31,2021-12-14,16:25,NaN,NaN,40.784615,-73.953964,"(40.784615, -73.953964)",EAST 93 STREET,NaN,NaN,...,14,16,POINT (-73.954 40.785),641261,E 94 St & Madison Ave,7344.01,40.785851,-73.954824,90,1
51,2022-04-24,15:35,MANHATTAN,10019.0,40.767242,-73.986206,"(40.767242, -73.986206)",WEST 56 STREET,9 AVENUE,NaN,...,24,15,POINT (-73.986 40.767),199470,W 54 St & 9 Ave,6920.05,40.766040,-73.987370,2034,1
51,2022-04-24,15:35,MANHATTAN,10019.0,40.767242,-73.986206,"(40.767242, -73.986206)",WEST 56 STREET,9 AVENUE,NaN,...,24,15,POINT (-73.986 40.767),591019,W 54 St & 9 Ave,6920.05,40.766040,-73.987370,374,1
66,2021-12-09,20:20,BROOKLYN,11223.0,40.592070,-73.962990,"(40.59207, -73.96299)",EAST 7 STREET,CRAWFORD AVENUE,NaN,...,9,20,POINT (-73.963 40.592),646032,Cortelyou Rd & Stratford Rd,2898.01,40.639660,-73.968070,118,1
66,2021-12-09,20:20,BROOKLYN,11223.0,40.592070,-73.962990,"(40.59207, -73.96299)",EAST 7 STREET,CRAWFORD AVENUE,NaN,...,9,20,POINT (-73.963 40.592),298836,Cortelyou Rd & Stratford Rd,2898.01,40.639660,-73.968070,116,1
72,2021-12-09,23:15,BROOKLYN,11218.0,40.640835,-73.989670,"(40.640835, -73.98967)",12 AVENUE,41 STREET,NaN,...,9,23,POINT (-73.99 40.641),803450,12 Ave & 36 St,3056.05,40.643546,-73.986418,42,1
72,2021-12-09,23:15,BROOKLYN,11218.0,40.640835,-73.989670,"(40.640835, -73.98967)",12 AVENUE,41 STREET,NaN,...,9,23,POINT (-73.99 40.641),299483,12 Ave & 36 St,3056.05,40.643546,-73.986418,46,1


#### Inspect the data

In [25]:
display(nypd_data.head())
print(nypd_data.shape)

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,year,month,day,hour
27,2021-12-14,12:54,BROOKLYN,11217.0,40.687534,-73.977500,"(40.687534, -73.9775)",FULTON STREET,SAINT FELIX STREET,NaN,...,4487052,Sedan,Bike,NaN,NaN,NaN,2021,12,14,12
31,2021-12-14,16:25,NaN,NaN,40.784615,-73.953964,"(40.784615, -73.953964)",EAST 93 STREET,NaN,NaN,...,4486581,Van,Bike,NaN,NaN,NaN,2021,12,14,16
51,2022-04-24,15:35,MANHATTAN,10019.0,40.767242,-73.986206,"(40.767242, -73.986206)",WEST 56 STREET,9 AVENUE,NaN,...,4521853,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN,2022,4,24,15
66,2021-12-09,20:20,BROOKLYN,11223.0,40.592070,-73.962990,"(40.59207, -73.96299)",EAST 7 STREET,CRAWFORD AVENUE,NaN,...,4485150,Bike,NaN,NaN,NaN,NaN,2021,12,9,20
72,2021-12-09,23:15,BROOKLYN,11218.0,40.640835,-73.989670,"(40.640835, -73.98967)",12 AVENUE,41 STREET,NaN,...,4485355,Sedan,Bike,NaN,NaN,NaN,2021,12,9,23


(58626, 33)


In [14]:
print(bike_data.shape)
display(bike_data.head())

(999338, 13)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,D86F678648E7A867,electric_bike,2024-09-10 22:50:16.212,2024-09-10 23:30:44.697,Hudson St & W 13 St,6115.06,Broadway & W 58 St,6948.10,40.740057,-74.005274,40.766953,-73.981693,casual
1,032D1788CD512084,electric_bike,2024-09-22 05:51:00.609,2024-09-22 05:56:50.446,W 37 St & 5 Ave,6398.06,9 Ave & W 45 St,6717.06,40.750380,-73.983390,40.760193,-73.991255,member
2,DA55381E5121F0F9,electric_bike,2024-09-24 11:07:40.618,2024-09-24 11:29:23.460,Greenpoint Ave & West St,5752.09,2 Ave & E 72 St,6925.09,40.729803,-73.959099,40.768762,-73.958408,member
3,F67A042C028C6367,classic_bike,2024-09-03 14:25:28.732,2024-09-03 14:33:51.075,E 85 St & 3 Ave,7212.05,2 Ave & E 72 St,6925.09,40.778012,-73.954071,40.768762,-73.958408,member
4,31F722D5EAB9C780,electric_bike,2024-09-09 15:46:50.376,2024-09-09 15:50:16.411,7 Ave & Park Pl,4125.07,Carroll St & 6 Ave,4019.06,40.677615,-73.973243,40.674089,-73.978728,member


#### Spatial

In [15]:
# Convert CitiBike start and end locations to GeoDataFrames
bike_start_gdf = gpd.GeoDataFrame(bike_data, geometry=gpd.points_from_xy(bike_data.start_lng, bike_data.start_lat))
bike_end_gdf = gpd.GeoDataFrame(bike_data, geometry=gpd.points_from_xy(bike_data.end_lng, bike_data.end_lat))


In [16]:
# Convert NYPD crash data to GeoDataFrame
nypd_spatial = nypd_data.dropna(subset=['LATITUDE', 'LONGITUDE'])
nypd_gdf = gpd.GeoDataFrame(nypd_spatial,
                            geometry=gpd.points_from_xy(nypd_spatial['LONGITUDE'], nypd_spatial['LATITUDE']))

In [17]:
bike_start_gdf = bike_start_gdf.set_crs("EPSG:2263")
bike_end_gdf = bike_end_gdf.set_crs("EPSG:2263")
nypd_gdf = nypd_gdf.set_crs("EPSG:2263")

In [21]:
nypd_spatial["LATITUDE"].unique()

array([40.687534, 40.784615, 40.767242, ..., 40.50958 , 40.625103,
       40.769352])

In [14]:
# Pernypd_gdfform spatial join to find crashes near CitiBike stations within 100 meters
nypd_near_start = gpd.sjoin_nearest(nypd_gdf, bike_start_gdf, distance_col="distance", max_distance=50)

NameError: name 'nypd_gdf' is not defined

In [ ]:
nypd_near_end = gpd.sjoin_nearest(nypd_gdf, bike_end_gdf, distance_col="distance", max_distance=1000)

In [ ]:
nypd_near_start.to_file("nypd_near_start.shp", driver="ESRI Shapefile")
nypd_near_end.to_file("nypd_near_end.shp", driver="ESRI Shapefile")

#### 1. Risk modeling and accident prediction

In [ ]:
# Define features and target for risk modeling
features = citibike_data[['start_hour', 'day_of_week', 'location_risk_score']]
target = merged_data['accident_occurrence']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

In [ ]:



import folium

# Create map with risk zones
map_nyc = folium.Map(location=[40.7128, -74.0060], zoom_start=12)
for _, row in high_risk_zones.iterrows():
    folium.CircleMarker(
        location=(row['lat'], row['lon']),
        radius=5,
        color='red',
        fill=True,
        fill_opacity=0.6
    ).add_to(map_nyc)
